In [1]:
from FlagEmbedding import BGEM3FlagModel # type: ignore
import json
import os
import numpy as np
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer
import re
useful_features = ["card_faces" , "cmc", "color_identity", "defense", "edhrec_rank", "game_changer", 
                   "keywords",  "life_modifier", "loyalty", "mana_cost", "name", "oracle_text", 
                   "power", "produced_mana", "toughness", "type_line", "rulings"]

saved_embeddings = "cardsDF_embeddings.pkl"
redo_embeddings = False

/home/ryan/miniconda3/envs/ml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#TODO # Need to be able to define the importance of words. 
# For example we get a lot of cards that are similar because they share the same "name " for example all of the urzas are more similar than other cards
# I want cards to similar soly their attributes. 
# (It may be better for now to just do the oracle text and ignore everything else? But these you still ahve the problem where cards reference themselves and the model thinks that means they are similar)
# Which yes they are however the name of the card should not matter. Only Some aspects of card text we care about
def gen_text_cards(df, summarize = True):
    samples = []
    for card_name, row in df.iterrows():
        #row_strings = [f"{card_name};; "]
        row_strings = [f""]
        for column in df.columns:
            if column != 'name' or column != card_name:
                value = row[column]
                #if column == 'oracle_text':
                #    #Replace the name on the card with [CARD_NAME]
                #    if np.char.find(value, card_name) != -1:
                #        value = np.char.replace(value, card_name, '[CARD_NAME]')

                #    name_split = re.split(' |,', card_name)
                #    if np.char.find(value, name_split[0]) != -1:
                #        value = np.char.replace(value, card_name[0], '[CARD_NAME]')
                if value == '':
                    continue
                row_strings.append(f"{column}; {value}")
        # Combine into single string
        #sample = f"{card_name}; " + ", ".join(row_strings)
        sample = f"".join(row_strings)
        samples.append(sample)
    
    return samples

In [3]:
if not os.path.exists(saved_embeddings):
    print(f"Building data frame and doing embeddings")
    with open("Data/oracle-cards-20250425090226.json", "r") as file:
        data = json.load(file)
    with open("Data/rulings-20250425090033.json", "r") as file:
        rulings = json.load(file)

    oracle_id_loc = {}
    for i in range(len(data)):
        oracle_id_loc[data[i]["oracle_id"]] = i
    
    for r in rulings:
        if 'rulings' not in data[oracle_id_loc[r["oracle_id"]]]:
            data[oracle_id_loc[r["oracle_id"]]]['rulings'] = r['comment'] 
        else: 
            data[oracle_id_loc[r["oracle_id"]]]['rulings'] + " " + r['comment']
    #Process the data
    df = pd.DataFrame(data, columns=useful_features)
    df = df[df['type_line'] != "Card // Card"]
    df = df.fillna('')
    df.set_index('name', inplace=True)
else:
    print(f"Dataframe read from {saved_embeddings}")
    df = pd.read_pickle(saved_embeddings)

Dataframe read from cardsDF_embeddings.pkl


In [4]:
#if redo_embeddings:
#    #Create Text card object to encode as sentences 
#    text_cards = gen_text_cards(df)
#    embedding_model = BGEM3FlagModel('BAAI/bge-m3') 
#    #card_embeddings = embedding_model.encode(df.index.tolist(), return_dense=True)
#    card_embeddings = embedding_model.encode(text_cards, return_dense=True)
#    print(card_embeddings)
#    print(df.shape, card_embeddings['dense_vecs'].shape)
#    print(card_embeddings['dense_vecs'][-1])
#
#    df['embeddings'] = card_embeddings['dense_vecs'].tolist()
#    df.to_pickle(saved_embeddings)
#    print(f"Build data frame and saved do {saved_embeddings}")

# Assume df is your DataFrame and saved_embeddings is the path to save embeddings
# Assume gen_text_cards(df) is a function you have defined to extract text

if redo_embeddings:
    # Create Text card object to encode as sentences
    text_cards = gen_text_cards(df)

    embedding_model = SentenceTransformer("/home/ryan/devel/projects/mtg-binder-ai/model")

    # Encode the text cards to get embeddings
    card_embeddings = embedding_model.encode(text_cards)

    print(card_embeddings)
    print(df.shape, card_embeddings.shape)  # SBERT directly returns the numpy array
    print(card_embeddings[-1])

    df['embeddings'] = card_embeddings.tolist()
    df.to_pickle(saved_embeddings)
    print(f"Build data frame and saved to {saved_embeddings}")

In [ ]:
print(text_cards[1])
print(df.columns)

Static Orb; , cmc; 3.0, color_identity; [], edhrec_rank; 4782.0, game_changer; False, keywords; [], mana_cost; {3}, oracle_text; As long as this artifact is untapped, players can't untap more than two permanents during their untap steps., type_line; Artifact, rulings; If there are two of these on the battlefield, they do not add together. The result is that only two permanents can be untapped., embeddings; [-0.04686294496059418, 0.019012443721294403, 0.054650623351335526, -0.048511262983083725, 0.023940781131386757, -0.06799487769603729, 0.044147882610559464, -0.06188139691948891, -0.014985217712819576, -0.008442436344921589, 0.019498400390148163, -0.027859942987561226, 0.001616023015230894, -0.021697739139199257, 0.0010911587160080671, 0.032624248415231705, -0.08643476665019989, 0.02758057788014412, -0.007558103185147047, 0.013699554838240147, 0.07038261741399765, -0.0379670187830925, -0.05699804797768593, -0.0116089703515172, 0.09013012796640396, 0.05539580434560776, -0.0630918890237

In [ ]:
embeddings = np.array(df["embeddings"].tolist()).astype('float32')
#
#string_to_int_id = {str_id: i for i, str_id in enumerate(df.index)}
##int_ids = np.array([string_to_int_id[idx] for idx in df.index]).astype('int64')
#
#index_flat = faiss.IndexFlatL2(embeddings.shape[1])
#index_flat.add(embeddings)
#
#res = faiss.StandardGpuResources()
#gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)

In [ ]:
print(df.head())

                         card_faces  cmc color_identity defense edhrec_rank  \
name                                                                          
Nissa, Worldsoul Speaker             4.0            [G]              9360.0   
Static Orb                           3.0             []              4782.0   
Sensory Deprivation                  1.0            [U]             26544.0   
Road of Return                       2.0            [G]              8232.0   
Storm Crow                           2.0            [U]             18314.0   

                          game_changer    keywords life_modifier loyalty  \
name                                                                       
Nissa, Worldsoul Speaker         False  [Landfall]                         
Static Orb                       False          []                         
Sensory Deprivation              False   [Enchant]                         
Road of Return                   False   [Entwine]                

In [ ]:
# Compute the most similar cards.
# Cosing similarity A*B/||A||*||B||. 
#   Dot product over the product of the maginitudes of the two vectors

#Inorder to find the most similar cards. \
#   compute the dot product of the query vector to every vector

# Recursive definition? can maybe you can compute the k most similar and then use the intersection to compute the similarity between all cards. might have a better run time. interesting experiment

# Better to compute as a matrix instead
from sklearn.metrics.pairwise import cosine_similarity
def single_soft_cosine(vAt, vBt):
    vA = vAt.reshape(1, -1)
    vB = vBt.reshape(1, -1)
    s = cosine_similarity(vA,vB)[0]
    neu = np.dot(vA, vB.reshape(-1,1)) * s
    d1 = np.sqrt(np.dot(vA, vA.reshape(-1,1))*s)
    d2 = np.sqrt(np.dot(vB, vB.reshape(-1,1))*s)
    #neu = np.dot(np.dot(vA, vB),s) 
    #d1 = np.sqrt(np.dot(np.dot(vA, vA),s))
    #d2 = np.sqrt(np.dot(np.dot(vB, vB),s))
    return neu/np.dot(d1, d2.reshape(-1,1))

#ssc_vec = np.vectorize(single_soft_cosine)
def soft_cosine(target, all_embeddings):
    #np.array(lambda x: target soft_cosine(target, x))
    return [single_soft_cosine(target, e) for e in all_embeddings]

    #return np.where(target != all_embeddings)
    #return np.array(single_soft_cosine(target, all_embeddings))
    #return ssc_vec(target, all_embeddings)

def cosine_similarity_search(df, card_name, all_embeddings, soft = True):
    if card_name not in df.index:
        print(f"{card_name} is not a card present within dataframe")
        return None
    else:
        target_embedding = np.array(df.loc[card_name, "embeddings"]).reshape(1, -1)
        if (not soft):
            similarities = cosine_similarity(target_embedding, all_embeddings)[0]
        else:
            similarities = soft_cosine(target_embedding, all_embeddings)

        similarities_df = pd.DataFrame({'card_name': df.index,
                                        'similarity': similarities})
        similarities_df = similarities_df[similarities_df['card_name'] != card_name]

        return similarities_df.sort_values(by='similarity', ascending=False)

In [ ]:
all_embeddings = np.stack(df["embeddings"].values)

#print(soft_cosine(all_embeddings[1], all_embeddings))

In [ ]:
#cos_search_results = cosine_similarity_search(df, "K'rrik, Son of Yawgmoth", all_embeddings)
cos_search_results = cosine_similarity_search(df, "Urza, Lord High Artificer", all_embeddings)
if cos_search_results is not None:
    print("Top 5 most similar cards to search")
    print(cos_search_results.head(10))
    for k in cos_search_results.head(10)["card_name"]:
        print(k + ":\n", df.loc[k]["oracle_text"], df.loc[k])


Top 5 most similar cards to search
                       card_name              similarity
9695                 Urza's Saga  [[0.7747412928062211]]
20082      Urza, Chief Artificer  [[0.7658522215695446]]
31317             Urza's Command  [[0.7536318956367878]]
13121    Codie, Vociferous Codex  [[0.7394258291788223]]
14405        Richlau, Headmaster  [[0.7339253664372426]]
797          Karn, Scion of Urza   [[0.731404619528387]]
22798  Mishra, Artificer Prodigy  [[0.7292800851325281]]
28009         Transmute Artifact  [[0.7273850451502981]]
13904            Artificer Class  [[0.7273399659472346]]
360    Urza's Construction Drone  [[0.7236056765266512]]
Urza's Saga:
 (As this Saga enters and after your draw step, add a lore counter. Sacrifice after III.)
I — This Saga gains "{T}: Add {C}."
II — This Saga gains "{2}, {T}: Create a 0/0 colorless Construct artifact creature token with 'This token gets +1/+1 for each artifact you control.'"
III — Search your library for an artifact card wi

In [ ]:
search_vect = embedding_model.encode("oracle_text")

In [ ]:
# Returns a string of words describing the cards role, each word is then accompanied by a weight to determine its main function or how "strong" the effect can be

def determine_draw_score(card_name, df):
    score = 0
    cmc = df.loc[card_name]["cmc"]

    for x in df.loc[card_name]["mana_cost"]:
        if x == "X":
            cmc += 1


    # a , each, X/Number, x is 1 by default

    #fo:"draw" (also some cards draw without drawing so fo:"put" fo:"a" fo:""
    num = 0
    splits = df.loc[card_name]["oracle_text"].split(" ")
    for i in range(len(splits)):
        word = splits[i]
        if word == "draw":
            if splits[i+1] == "a":
                num += 1
                i += 1
                if splits[i] == "for":
                    if splits[i+1] == "each":
                        if splits[i+2] == "opponenet":
                            num += 2

            elif splits[i+1].isnumeric() and int(splits[i+1]):
                num += (int(splits[i+1]))
                i += 1
                if splits[i] == "for":
                    if splits[i+1] == "each":
                        if splits[i+2] == "opponenet":
                            num *= 2
        

    #For each opponent = 3
    # parse looking for key phrases
    return num/cmc
# Draw, Mana, Control, Lethality, ThemedSupprt 
#Can I use a transformer for self attention refering to the themed support of the card? I.e themed supprt is zombies and then assign the score based on how well,
#   the card supports that theme
def determine_card_role_scores(card_name, df):
    #return "Draw:, Mana:, Control:, Lethality:"
    #draw_score, mana_score, control_score, lethality_score = 0
    # draw score = numcards/manaspent
    # mana_score = manacreated/manaspent
    # control_score = objectsAffected/manaspent
    # lehality_score = numplayers_affected * (damage/mana)/40 (replace damage with mill and 40 with 100 for mill)

    #Maybe I can Use some scryfall syntax stuff here? 
    draw_score = determine_draw_score(card_name, df)
    return (draw_score, mana_score, control_score, lethality_score)

In [ ]:
df.loc["Urza, Lord High Artificer"]['oracle_text']

'When Urza enters, create a 0/0 colorless Construct artifact creature token with "This token gets +1/+1 for each artifact you control."\nTap an untapped artifact you control: Add {U}.\n{5}: Shuffle your library, then exile the top card. Until end of turn, you may play that card without paying its mana cost.'

In [ ]:

#import matplotlib.pyplot as plt
#import numpy as np
#import hdbscan
#import faiss
#from mpl_toolkits.mplot3d import Axes3D
#from sklearn.decomposition import PCA

#num_random = 1000
#random_samp = [0]
#possible_indices = np.arange(1, embeddings.shape[0])
#random_indices = np.random.choice(possible_indices, size=num_random, replace=False)
#random_samp.extend(random_indices)

#r_embeddings = embeddings[random_samp]


#r_embeddings = embeddings[:500]

#index_flat = faiss.IndexFlatL2(r_embeddings.shape[1])
#index_flat.add(r_embeddings)

#res = faiss.StandardGpuResources()
#gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)

#query_index = 0  
#query_vector_original = r_embeddings[query_index].reshape(1, -1)

##knn
#k = 20  
#distances, indices = gpu_index_flat.search(query_vector_original, k)
#nearest_neighbor_vectors_original = r_embeddings[indices[0]]

## Apply PCA for Dimensionality Reduction to 3D
#n_components = 3
#pca = PCA(n_components=n_components)
#reduced_embeddings = pca.fit_transform(r_embeddings)
#reduced_query_vector = pca.transform(query_vector_original)[0]
#reduced_nearest_neighbors = pca.transform(nearest_neighbor_vectors_original)

##HDBScan
#clusterer = hdbscan.HDBSCAN(min_cluster_size=3)
#clusters = clusterer.fit_predict(reduced_embeddings)
##clusters = clusterer.fit_predict(r_embeddings)

#fig = plt.figure(figsize=(10, 12))
#ax = fig.add_subplot(111, projection='3d')

##ax.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], reduced_embeddings[:, 2], color='black', label='Data Points (PCA Reduced)', alpha=0.4)
#ax.scatter(reduced_query_vector[0], reduced_query_vector[1], reduced_query_vector[2], color='red', marker='x', s=100, label="Nissa, Worldsoul Speaker")
#ax.scatter(reduced_nearest_neighbors[:, 0], reduced_nearest_neighbors[:, 1], reduced_nearest_neighbors[:, 2], color='orange', s=80, label='Nearest Neighbors (PCA Reduced)', alpha=0.5)
#unique_clusters = np.unique(clusters)
##for label, color in zip(np.unique(unique_clusters), colors):
##    cluster_mask = (clusters == label)
##    ax.scatter(reduced_embeddings[cluster_mask, 0],
##               reduced_embeddings[cluster_mask, 1],
##               reduced_embeddings[cluster_mask, 2],
##               c=[color],  # Use a list to ensure color is applied correctly
##               marker='o',
##               alpha=1,)
#unique_labels = np.unique(clusters)
#cmap = plt.cm.viridis
## Plot each cluster
#for label in unique_labels:
    #cluster_mask = (clusters == label)
    #if label == -1:
        ## Outliers: make them black
        #ax.scatter(reduced_embeddings[cluster_mask, 0],
                   #reduced_embeddings[cluster_mask, 1],
                   #reduced_embeddings[cluster_mask, 2],
                   #c='black',
                   #marker='o',
                   #alpha=1,
                   #label='Outliers')
    #else:
        ## Cluster members: use the colormap
        ## Find the index of the current label in the sorted unique labels (excluding -1)
        #sorted_labels = np.sort(unique_labels[unique_labels != -1])
        #if len(sorted_labels) > 0:
            #color_index = np.where(sorted_labels == label)[0][0]
            #color = cmap(color_index / (len(sorted_labels) - 1) if len(sorted_labels) > 1 else cmap(0))
        #else:
            #color = cmap(0) # Default color if no clusters other than outliers

        #ax.scatter(reduced_embeddings[cluster_mask, 0],
                   #reduced_embeddings[cluster_mask, 1],
                   #reduced_embeddings[cluster_mask, 2],
                   #c=[color],
                   #marker='o',
                   #alpha=1
                   #)
#ax.set_title('3D Visualization of Vector Database Search (PCA Reduced)')
#ax.legend(loc='lower right')

## Annotate the nearest neighbors (optional)
##for i, neighbor in enumerate(reduced_nearest_neighbors):
##    ax.text(neighbor[0], neighbor[1], neighbor[2], f'NN {i+1}', fontsize=9)

## Show the plot
#plt.show()